<center>

<h1 style="text-align:center"> Generalized Algebraic Data Types </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Simple language

Consider this simple language of integers and booleans

In [357]:
type value =
  | Int of int
  | Bool of bool

type expr =
  | Val of value
  | Plus of expr * expr
  | Mult of expr * expr
  | Ite of expr * expr * expr

type value = Int of int | Bool of bool


type expr =
    Val of value
  | Plus of expr * expr
  | Mult of expr * expr
  | Ite of expr * expr * expr


## Evaluator for the simple language

We can write a simple evaluator for this language

In [358]:
let rec eval : expr -> value =
  fun e -> match e with
  | Val (Int i) -> Int i
  | Val (Bool i) -> Bool i
  | Plus (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 + i2)
  | Mult (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 * i2)
  | Ite (p,e1,e2) ->
    let Bool b = eval p in
    if b then eval e1 else eval e2  

File "[358]", line 6, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))
File "[358]", line 9, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))
File "[358]", line 12, characters 4-61:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Int _


val eval : expr -> value = <fun>


## Evaluator for the simple language

* The compiler complains that programs such as `true + 10` is not handled.
  + Our evaluator gets **stuck** when it encouters such an expression.

In [3]:
eval @@ Plus (Val (Bool true), Val (Int 10))

error: runtime_error

* We need **Types**
  + Well-typed programs do not get stuck!

## Phantom types

* We can add types to our values using a technique called **phantom types**

In [361]:
type 'a value =
  | Int of int
  | Bool of bool

type 'a value = Int of int | Bool of bool


* Observe that `'a` only appears on the LHS.
  + This `'a` is called a phantom type variable.
* What is this useful for?

## Typed expression language

We can add types to our expression language now using phantom type

In [362]:
type 'a expr =
  | Val of 'a value
  | Plus of int expr * int expr
  | Mult of int expr * int expr
  | Ite of bool expr * 'a expr * 'a expr

(* Assign concerte type to the phantom type variable 'a*)
let mk_int i : int expr = Val (Int i)
let mk_bool b : bool expr = Val (Bool b)
let plus e1 e2 : int expr = Plus (e1, e2)
let mult e1 e2 : int expr = Mult (e1, e2)

type 'a expr =
    Val of 'a value
  | Plus of int expr * int expr
  | Mult of int expr * int expr
  | Ite of bool expr * 'a expr * 'a expr


val mk_int : int -> int expr = <fun>


val mk_bool : bool -> bool expr = <fun>


val plus : int expr -> int expr -> int expr = <fun>


val mult : int expr -> int expr -> int expr = <fun>


## Benefit of phantom types

In [363]:
let i = Val (Int 0);;
let i' = mk_int 0;;

let b = Val (Bool true);;
let b' = mk_bool true;;

let p = Plus (i,i);;
let p' = plus i i;;

val i : 'a expr = Val (Int 0)


val i' : int expr = Val (Int 0)


val b : 'a expr = Val (Bool true)


val b' : bool expr = Val (Bool true)


val p : 'a expr = Plus (Val (Int 0), Val (Int 0))


val p' : int expr = Plus (Val (Int 0), Val (Int 0))


## Benefit of phantom types

We no longer allow ill-typed expression if we use the helper functions.

In [364]:
plus (mk_bool true) (mk_int 10) 

error: compile_error

## Typed evaluator

We can write an evaluator for this language now.

Let's use the same evaluator as the earlier one.

In [365]:
let rec eval : 'a expr -> 'a value = 
  fun e -> match e with
  | Val (Int i) -> Int i
  | Val (Bool i) -> Bool i
  | Plus (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 + i2)
  | Mult (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 * i2)
  | Ite (p,e1,e2) ->
    let Bool b = eval p in
    if b then eval e1 else eval e2

File "[365]", line 6, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))
File "[365]", line 9, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))


error: compile_error

## Typed evaluator

* We see a $\color{red}{\text{type error}}$.
* OCaml by default expects the function expression at the recursive call position to have the same type as the outer function.
* This need not be the case if the function is polymorphic
  + `eval (p : int expr)` and `eval (p : bool expr)`.
* In order to allow this, OCaml supports polymorphic recursion (aka Milner-Mycroft typeability)
  + Robin Milner co-invented type infererence + polymorphism that we using in OCaml.
  + Alan Mycroft was my mentor at Cambridge :-) 

## Fixing the interpreter with polymorphic recursion

`type a` is known as **locally abstract type**.

In [9]:
let rec eval : type a. a expr -> a value = 
  fun e -> match e with
  | Val (Int i) -> Int i
  | Val (Bool i) -> Bool i
  | Plus (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 + i2)
  | Mult (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 * i2)
  | Ite (p,e1,e2) ->
    let Bool b = eval p in
    if b then eval e1 else eval e2

File "[9]", line 6, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))
File "[9]", line 9, characters 4-62:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Int _, Bool _)|(Bool _, _))
File "[9]", line 12, characters 4-61:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Int _


val eval : 'a expr -> 'a value = <fun>


## Errors gone, but warning remains

* Compiler still warns us that there are unhandled cases in pattern matches
* But haven't we added types to the expression language?
* Observe that `mk_int i = Val (Int i)` is just convention.
  + You can still write ill-typed expression by directly using the constructors.

In [10]:
eval @@ Plus (Val (Bool true), Val (Int 10))

error: runtime_error

* Here, `Bool true` is inferred to have the type `int value`.
  + Need a way to inform the compiler that `Bool true` has type `bool value`.

## Generalized Algebraic Data Types

GADTs allow us to **refine** the return type of the data constructor. 

In [39]:
type 'a value =
  | Int : int -> int value
  | Bool : bool -> bool value
  
type 'a expr =
  | Val : 'a value -> 'a expr
  | Plus : int expr * int expr -> int expr
  | Mult : int expr * int expr -> int expr
  | Ite : bool expr * 'a expr * 'a expr -> 'a expr

type 'a value = Int : int -> int value | Bool : bool -> bool value


type 'a expr =
    Val : 'a value -> 'a expr
  | Plus : int expr * int expr -> int expr
  | Mult : int expr * int expr -> int expr
  | Ite : bool expr * 'a expr * 'a expr -> 'a expr


## Evaluator remains the same

Observe that the warnings are also gone!

In [41]:
let rec eval : type a. a expr -> a value = 
  fun e -> match e with
  | Val (Int i) -> Int i
  | Val (Bool i) -> Bool i
  | Plus (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 + i2)
  | Mult (e1, e2) ->
    let Int i1, Int i2 = eval e1, eval e2 in
    Int (i1 * i2)
  | Ite (p,e1,e2) ->
    let Bool b = eval p in
    if b then eval e1 else eval e2

val eval : 'a expr -> 'a value = <fun>


In [13]:
eval @@ Plus (Val (Bool true), Val (Int 10))

error: runtime_error

## Absurd types

GADTs don't prevent you from instantiating absurd types. Consider

```ocaml
type 'a value =
  | Int : int -> int value
  | Bool : bool -> bool value
```

In [130]:
type t = string value

type t = string value


* There is no term with type `string value`.
* We will ignore such types.

## GADTs are very powerful!

* Features
  + Refine return types.
  + Introduce existential types.

* Some uses
  + Typed domain specific languages
    * The example that we just saw...
  + (Lightweight) dependently typed programming
    * Enforcing shape properties of data structures
  + Generic programming
    * Implementing functions like `map` and `fold` operate on the shape of the data once and for all!

## Units of measure

* In 1999, $125 million mars climate orbiter was lost due to units of measurement error
  + Lockheed Martin used Imperial and NASA used Metric
  + Use GADTs to avoid such errors, but still host both units of measure in the same program

## Units of measure

In [278]:
type kelvin
type celcius
type farenheit

type _ temp = 
  | Kelvin : float -> kelvin temp
  | Celcius : float -> celcius temp
  | Farenheit : float -> farenheit temp 

type kelvin


type celcius


type farenheit


type _ temp =
    Kelvin : float -> kelvin temp
  | Celcius : float -> celcius temp
  | Farenheit : float -> farenheit temp


## Units of measure

In [280]:
let add_temp : type a. a temp -> a temp -> a temp =
  fun a b -> match a,b with
  | Kelvin a, Kelvin b -> Kelvin (a+.b)
  | Celcius a, Celcius b -> Celcius (a+.b)
  | Farenheit a, Farenheit b -> Farenheit (a+.b)

val add_temp : 'a temp -> 'a temp -> 'a temp = <fun>


In [281]:
add_temp (Kelvin 20.23) (Kelvin 30.5)

- : kelvin temp = Kelvin 50.730000000000004


In [282]:
add_temp (Kelvin 20.23) (Celcius 12.3)

error: compile_error

## Church Numerals

* Many of the shape properties of data structure involve some integers. 
* Let's encode natural numbers using GADTs.
  + This is the familiar Church encoding.

In [2]:
type z = Z
type 'n s = S : 'n -> 'n s

type z = Z


type 'n s = S : 'n -> 'n s


In [16]:
S (S Z)

- : z s s = S (S Z)


## Length-indexed lists

Some of the list function in the OCaml list library as quite unsatisfying.

In [5]:
List.hd []

error: runtime_error

In [17]:
List.tl []

error: runtime_error

## Length indexed lists

* Let's implement our own list type which will statically catch these errors.
* The idea is to encode the **length** of the list in the **type** of the list.
  + Use our encoding of church numerals.

In [11]:
type (_,_) list =
  | Nil  : ('a, z ) list
  | Cons : 'a * ('a,'n) list -> ('a, 'n s) list

type (_, _) list =
    Nil : ('a, z) list
  | Cons : 'a * ('a, 'n) list -> ('a, 'n s) list


In [12]:
Nil;;
Cons(0,Nil);;
Cons(0,Cons(1,Nil));;

- : ('a, z) list = Nil


- : (int, z s) list = Cons (0, Nil)


- : (int, z s s) list = Cons (0, Cons (1, Nil))


## Safe `hd` and `tl`

Define the function `hd` and `tl` such that they can only be applied to non-empty lists.

In [13]:
let hd (l : ('a,'n s) list) : 'a = 
  let Cons (v,_) = l in
  v

val hd : ('a, 'n s) list -> 'a = <fun>


In [14]:
hd (Cons (1, Nil))

- : int = 1


In [15]:
hd Nil

error: compile_error

## Safe `hd` and `tl`

Define the function `hd` and `tl` such that they can only be applied to non-empty lists.

In [23]:
let hd (l : ('a,'n s) list) : 'a = 
  let Cons (x,_) = l in
  x

val hd : ('a, 'n s) list -> 'a = <fun>


* Observe that OCaml does not complain about `Nil` case not handled.
  + Does not apply since `l` is non-empty!
  + GADTs allow the compiler to refute cases statically
    * Generate more efficient code!

## Safe `hd` and `tl`

In [24]:
let tl (l : ('a,'n s) list) : ('a, 'n) list = 
  let Cons (_,xs) = l in
  xs

val tl : ('a, 'n s) list -> ('a, 'n) list = <fun>


In [25]:
tl (Cons (0, Cons(1,Nil)));;
tl (Cons (0, Nil));;

- : (int, z s) list = Cons (1, Nil)


- : (int, z) list = Nil


## List map

`map` is length preserving

In [26]:
let rec map : type n. ('a -> 'b) -> ('a, n) list -> ('b, n) list =
  fun f l ->
    match l with
    | Nil -> Nil
    | Cons (x,xs) -> Cons(f x, map f xs)

val map : ('a -> 'b) -> ('a, 'n) list -> ('b, 'n) list = <fun>


## List.rev

Tricky to implement tail recurive list like:

```ocaml
let rec rev l acc =
  match l with
  | Nil -> acc
  | Cons(x,xs) -> rev xs (Cons (x,acc))
```

why?

The type of this function is 

```ocaml
('a,'n) list -> ('a,'m) list -> ('a, 'm + 'n) list
```

We don't have type-level arithmetic

## List.rev 

Here is another attempt:

In [83]:
let rec app1 : type n. ('a, n) list -> 'a -> ('a, n s) list =
  fun l v ->
    match l with
    | Nil -> Cons (v, Nil)
    | Cons(x,xs) -> Cons (x, app1 xs v)
    
let rec rev : type n. ('a, n) list -> ('a, n) list =
  fun l ->
    match l with
    | Nil -> Nil
    | Cons(x,xs) -> app1 (rev xs) x

val app1 : ('a, 'n) list -> 'a -> ('a, 'n s) list = <fun>


val rev : ('a, 'n) list -> ('a, 'n) list = <fun>


In [84]:
rev (Cons (0, Cons (1, Cons (2, Nil))))

- : (int, z s s s) list = Cons (2, Cons (1, Cons (0, Nil)))


## Type level addition

* We observed that tail recursive list function requires type level addition on church numerals
* OCaml doesn't support type level functions natively
  + But we can construct **proofs** for type level functions.

## Type level addition

In [204]:
type (_,_,_) plus = 
  | PlusZero : (z,'n,'n) plus
  | PlusShift  : ('a, 'b s, 'c s) plus -> ('a s, 'b, 'c s) plus

type (_, _, _) plus =
    PlusZero : (z, 'n, 'n) plus
  | PlusShift : ('a, 'b s, 'c s) plus -> ('a s, 'b, 'c s) plus


* `('a, 'b, 'c) plus` $\equiv$ `'c = 'a + 'b` where `'a, 'b, 'c` are type level church numerals.
* `PlusZero` and `PlusOne` are theorems on numbers.
  + $0 + n = n$
  + $a + (b + 1) = c+1 \implies (a + 1) + b = c + 1$

## Type level addition

In [204]:
type (_,_,_) plus = 
  | PlusZero : (z,'n,'n) plus
  | PlusShift  : ('a, 'b s, 'c s) plus -> ('a s, 'b, 'c s) plus

type (_, _, _) plus =
    PlusZero : (z, 'n, 'n) plus
  | PlusShift : ('a, 'b s, 'c s) plus -> ('a s, 'b, 'c s) plus


* From a Curry-Howard perspective, we can view a value of type `(a,b,c) plus` as a proof of the proposition that `a + b = c`
* Looked at this way, `plus` and other GADTs are a convenient way of programming with proofs as first-class objects.

## Tail recursive reverse

In [206]:
let rec rev : type m n o. (m,n,o) plus -> ('a, m) list -> ('a, n) list -> ('a, o) list =
  fun p l acc ->
    match p, l with
    | PlusZero, Nil -> acc
    | PlusShift p', Cons(x,xs) ->  rev p' xs (Cons(x,acc))

val rev :
  ('m, 'n, 'o) plus -> ('a, 'm) list -> ('a, 'n) list -> ('a, 'o) list =
  <fun>


## Tail recursive reverse

In [216]:
let proof = PlusShift (PlusShift (PlusShift (PlusZero : (z,z s s s,z s s s) plus)))

val proof : (z s s s, z, z s s s) plus =
  PlusShift (PlusShift (PlusShift PlusZero))


In [218]:
rev proof (Cons (0, Cons (1, Cons (2, Nil)))) Nil

- : (int, z s s s) list = Cons (2, Cons (1, Cons (0, Nil)))


## Trees

Here is an unconstrained tree data type:

In [49]:
type 'a tree = 
  | Empty
  | Tree of 'a tree * 'a * 'a tree

type 'a tree = Empty | Tree of 'a tree * 'a * 'a tree


In [194]:
Tree (Empty, 1, Tree (Empty, 2, Tree (Empty, 3, Empty)));; (* Right skewed *)
Tree (Tree (Tree (Empty, 3, Empty), 2, Empty), 1, Empty);; (* Left skewed *)
Tree (Tree (Tree (Empty, 3, Empty), 2, Tree (Empty, 3, Empty)), 
      1, 
      Tree (Tree (Empty, 3, Empty), 2, Tree (Empty, 3, Empty))) (* Perfectly balanced tree *)

- : int tree = Tree (Empty, 1, Tree (Empty, 2, Tree (Empty, 3, Empty)))


- : int tree = Tree (Tree (Tree (Empty, 3, Empty), 2, Empty), 1, Empty)


- : int tree =
Tree (Tree (Tree (Empty, 3, Empty), 2, Tree (Empty, 3, Empty)), 1,
 Tree (Tree (Empty, 3, Empty), 2, Tree (Empty, 3, Empty)))


## Tree operations

In [56]:
let rec depth t = match t with
  | Empty -> 0
  | Tree (l,_,r) -> 1 + max (depth l) (depth r)

val depth : 'a tree -> int = <fun>


In [57]:
let top t = match t with
  | Empty -> None 
  | Tree (_,v,_) -> Some v

val top : 'a tree -> 'a option = <fun>


`swivel` is mirror image of the tree

In [58]:
let rec swivel t = match t with
  | Empty -> Empty
  | Tree (l,v,r) -> Tree (swivel r, v, swivel l)

val swivel : 'a tree -> 'a tree = <fun>


## Perfectly Balanced Trees via nesting

In [60]:
type 'a ntree = 
  | EmptyN
  | TreeN of 'a * ('a *'a) ntree 

type 'a ntree = EmptyN | TreeN of 'a * ('a * 'a) ntree


In [68]:
TreeN (1, TreeN((2,2), TreeN (((3,3),(3,3)),EmptyN)))

- : int ntree = TreeN (1, TreeN ((2, 2), TreeN (((3, 3), (3, 3)), EmptyN)))


## Operations on `ntree`

In [73]:
let rec depthN : type a. a ntree -> int = 
  fun t -> match t with
  | EmptyN -> 0
  | TreeN (_,t') -> 1 + depthN t'

val depth : 'a ntree -> int = <fun>


In [75]:
let topN t = match t with
  | EmptyN -> None 
  | TreeN (v,_) -> Some v

val top : 'a ntree -> 'a option = <fun>


In [81]:
let rec swivelN : type a. (a -> a) -> a ntree -> a ntree =
  fun f t -> match t with
  | EmptyN -> EmptyN
  | TreeN (v,t') -> TreeN (f v, swivelN (fun (x,y) -> (f x, f y)) t')
  
let swivelN t = swivelN (fun x -> x) t

val swivelN : ('a -> 'a) -> 'a ntree -> 'a ntree = <fun>


val swivelN : 'a ntree -> 'a ntree = <fun>


## Perfectly balanced tree using GADTs

In [85]:
type ('a,_) gtree = 
  | EmptyG : ('a,z) gtree
  | TreeG  : ('a,'n) gtree * 'a * ('a,'n) gtree -> ('a,'n s) gtree

type ('a, _) gtree =
    EmptyG : ('a, z) gtree
  | TreeG : ('a, 'n) gtree * 'a * ('a, 'n) gtree -> ('a, 'n s) gtree


In [86]:
TreeG (TreeG (TreeG (EmptyG, 3, EmptyG), 2, TreeG (EmptyG, 3, EmptyG)), 
       1, 
       TreeG (TreeG (EmptyG, 3, EmptyG), 2, TreeG (EmptyG, 3, EmptyG)))

- : (int, z s s s) gtree =
TreeG (TreeG (TreeG (EmptyG, 3, EmptyG), 2, TreeG (EmptyG, 3, EmptyG)), 1,
 TreeG (TreeG (EmptyG, 3, EmptyG), 2, TreeG (EmptyG, 3, EmptyG)))


## Operations on `gtree`

In [91]:
let rec depthG : type n. ('a,n) gtree -> int =
  fun t -> match t with
  | EmptyG -> 0
  | TreeG (l,_,_) -> 1 + depthG l

val depthG : ('a, 'n) gtree -> int = <fun>


In [97]:
let topG : ('a, 'n s) gtree -> 'a = 
  fun t -> let TreeG(_,v,_) = t in v

val topG : ('a, 'n s) gtree -> 'a = <fun>


In [98]:
let rec swivelG : type n.('a,n) gtree -> ('a,n) gtree = 
fun t -> match t with
    EmptyG -> EmptyG
  | TreeG (l,v,r) -> TreeG (swivelG r, v, swivelG l)

val swivelG : ('a, 'n) gtree -> ('a, 'n) gtree = <fun>


## Zipping perfect trees

In [100]:
let rec zipTree :
  type n.('a,n) gtree -> ('b,n) gtree -> ('a * 'b,n) gtree =
  fun x y -> match x, y with
      EmptyG, EmptyG -> EmptyG
    | TreeG (l,v,r), TreeG (m,w,s) ->
      TreeG (zipTree l m, (v,w), zipTree r s)

val zipTree : ('a, 'n) gtree -> ('b, 'n) gtree -> ('a * 'b, 'n) gtree = <fun>


## Depth indexed tree (unbalanced)

```ocaml
type ('a,_) dtree =
  | EmptyD : ('a,z) dtree
  | TreeD : ('a,'n) dtree * 'a * ('a,'n) dtree 
            * ('m,'n,'o) max -> ('a,'o s) dtree
```

* The type `('m,'n,'o) max` $~\equiv~$ `max('m,'n) = 'o` on type level church numerals.
  + `('m,'n,'o) max` is **proof** that `'o` is the max of `'m` and `'n`.
* `TreeD` carries a value of type `('m,'n,'o) max`
  + This is **proof carrying code**
  + Given `t = TreeD (l,v,r,p : ('m,'n,'o) max)`, we can prove that `t` has depth `'o s`.

## Equality GADT

For defining `max` start with the **Equality GADT**.

The equality GADT is the type:

In [127]:
type (_,_) eql = Refl : ('a,'a) eql

type (_, _) eql = Refl : ('a, 'a) eql


We can only instantiate `Refl` with types that are known to be equal.

In [128]:
type t = int
let _ = (Refl : (t,int) eql)

type t = int


- : (t, int) eql = Refl


## Equality GADT

`Refl` cannot be instantiated at types known to be different or not known to be equal.

In [125]:
let _ = (Refl : (int,string) eql)

error: compile_error

In [126]:
module M : sig type t end = struct type t = int end
let _ = (Refl : (M.t,int) eql)

module M : sig type t end


error: compile_error

## `max` type

The following type definitions contain theorems about the `max` function.

In [129]:
type (_,_,_) max =
    MaxEq : ('a,'b) eql -> ('a,'b,'a) max
  | MaxFlip : ('a,'b,'c) max -> ('b,'a,'c) max
  | MaxSuc : ('a,'b,'a) max -> ('a s,'b,'a s) max

type (_, _, _) max =
    MaxEq : ('a, 'b) eql -> ('a, 'b, 'a) max
  | MaxFlip : ('a, 'b, 'c) max -> ('b, 'a, 'c) max
  | MaxSuc : ('a, 'b, 'a) max -> ('a s, 'b, 'a s) max


## `max` type examples

In [156]:
let m1 = MaxEq (Refl : (z, z) eql)

val m1 : (z, z, z) max = MaxEq Refl


In [157]:
let m2 = MaxSuc m1

val m2 : (z s, z, z s) max = MaxSuc (MaxEq Refl)


Given a proof that the max of `z` and `z` is `z`, I can prove that the max of `z s` and `z` is `z s`.

## `max` function on church numerals

In [167]:
let rec max : type a b c.(a,b,c) max -> a -> b -> c
  = fun mx m n -> match mx,m with
      MaxEq Refl , _    -> m
    | MaxFlip mx', _    -> max mx' n m
    | MaxSuc mx' , S m' -> S (max mx' m' n)

val max : ('a, 'b, 'c) max -> 'a -> 'b -> 'c = <fun>


In [166]:
max (MaxSuc (MaxEq (Refl : (z,z) eql))) (S Z) Z

- : z s = S Z


## `max` function on church numerals

In [165]:
max (MaxSuc (MaxEq (Refl : (z,z) eql))) (S Z) Z

- : z s = S Z


* The `max` function seems a bit silly given that we need to provide the proof.
  + We learn no new information. Proof already tells us that `z s > z`.
* `max` function gets **term** level evidence from the **type** level proof. 

## Depth Indexed Tree

In [168]:
type ('a,_) dtree =
    EmptyD : ('a,z) dtree
  | TreeD : ('a,'m) dtree * 'a * ('a,'n) dtree * ('m,'n,'o) max 
    -> ('a,'o s) dtree

type ('a, _) dtree =
    EmptyD : ('a, z) dtree
  | TreeD : ('a, 'm) dtree * 'a * ('a, 'n) dtree *
      ('m, 'n, 'o) max -> ('a, 'o s) dtree


## Operations on `dtree`

In [219]:
let rec depthD : type a n.(a,n) dtree -> n = function
    EmptyD -> Z
  | TreeD (l,_,r,mx) -> S (max mx (depthD l) (depthD r))

val depthD : ('a, 'n) dtree -> 'n = <fun>


In [220]:
let topD : type a n.(a,n s) dtree -> a =
  function TreeD (_,v,_,_) -> v

val topD : ('a, 'n s) dtree -> 'a = <fun>


In [221]:
let rec swivelD :
  type a n.(a,n) dtree -> (a,n) dtree = function
    EmptyD -> EmptyD
  | TreeD (l,v,r,m) ->
    TreeD (swivelD r, v, swivelD l, MaxFlip m)

val swivelD : ('a, 'n) dtree -> ('a, 'n) dtree = <fun>


## Existentials 

* We had studied existentials in System F.
* GADTs also introduce existential types.

In [222]:
type t = Pack : 'a -> t

type t = Pack : 'a -> t


* Observe that the `'a` does not appear on the RHS.
  + `'a` is the existential type.

## Existential List

With existentials you can create list that contains values of different types.

In [235]:
[Pack 10; Pack "Hello"; Pack true]

- : t list = [Pack <poly>; Pack <poly>; Pack <poly>]


* This particular list isn't useful
  + Given `Pack v`, we only know that `v` has some type `'a`.
  + We do not have any useful operations on values of type `'a`; it is too polymorphic. 

## Existential list : showable

Here is a more useful heterogeneous list: List of printable values.

In [246]:
type showable = Showable : 'a * ('a -> string) -> showable

type showable = Showable : 'a * ('a -> string) -> showable


In [244]:
let l = [Showable (10, string_of_int); Showable ("Hello", fun x -> x); 
         Showable (3.14, string_of_float)]

val l : showable list =
  [Showable (<poly>, <fun>); Showable (<poly>, <fun>);
   Showable (<poly>, <fun>)]


In [242]:
List.map (fun (Showable (v,show)) -> show v) l

- : string list = ["10"; "Hello"; "3.14"]


## GADTs and Modules

The type `type showable = Showable : 'a * ('a -> string) -> showable` is equivalent to

```ocaml
module type Showable : sig
  type t
  val value : t
  val show : t -> string
end
```

And the value `Showable (10, string_of_int)` is equivalent to


```ocaml
module IntShowable : Showable = struct 
  type t = int
  let value = 10
  let show = string_of_int
end
```

Both GADTs and Modules introduce existentials.

## Encoding Pairs

* Recall from our lambda calculus lectures the type `pair` 
  + $\langle M : A, N : B \rangle : A \times B$
* These are similar to OCaml tuple data type, but limited to two elements.
  + `(10, false, 10.4)` will become $\langle 10, \langle false, 10.4 \rangle \rangle$.
* We can encode pairs using GADTs

## Encoding Pairs 

In [275]:
type _ hlist =
  | Nil : z hlist
  | Cons : 'a * 'b hlist -> ('a * 'b) hlist

type _ hlist = Nil : z hlist | Cons : 'a * 'b hlist -> ('a * 'b) hlist


In [259]:
let l = Cons (10, Cons (false, Cons (10.4, Nil)))

val l : (int * (bool * (float * z))) hlist =
  Cons (10, Cons (false, Cons (10.4, Nil)))


## Encoding Pairs : Accessor Functions

In [274]:
let fst : ('a * _) hlist -> 'a = fun (Cons (x,_)) -> x
let snd : (_ * ('a * _)) hlist -> 'a = fun (Cons (_,Cons(x,_))) -> x
let trd : (_ * (_ * ('a * _))) hlist -> 'a = fun (Cons(_,Cons (_,Cons(x,_)))) -> x

val fst : ('a * 'b) hlist -> 'a = <fun>


val snd : ('b * ('a * 'c)) hlist -> 'a = <fun>


val trd : ('b * ('c * ('a * 'd))) hlist -> 'a = <fun>


In [273]:
trd (Cons (10, Cons (true, Cons(10.5, Nil))))

- : float = 10.5


<center>

<h1 style="text-align:center"> Fin. </h1>
</center>

<center>

<h1 style="text-align:center"> Extra Materials. </h1>
</center>

## Adding lambdas to interpreter

Let's add simply typed lambda calculus to our original int & bool expression evaluator.

In [324]:
type _ typ = 
  | TInt : int typ
  | TBool : bool typ
  | TLam : 'a typ * 'b typ -> ('a -> 'b) typ

type 'a value =
  | VInt : int -> int value
  | VBool : bool -> bool value
  | VLam : ('a value -> 'b value) -> ('a -> 'b) value
  
and 'a expr =
  | Var : string * 'a typ ->  'a expr
  | App : ('a -> 'b) expr * 'a expr -> 'b expr
  | Lam : string * 'a typ * 'b expr -> ('a -> 'b) expr
  | Val : 'a value -> 'a expr
  | Plus : int expr * int expr -> int expr
  | Mult : int expr * int expr -> int expr
  | Ite : bool expr * 'a expr * 'a expr -> 'a expr

type _ typ =
    TInt : int typ
  | TBool : bool typ
  | TLam : 'a typ * 'b typ -> ('a -> 'b) typ


type 'a value =
    VInt : int -> int value
  | VBool : bool -> bool value
  | VLam : ('a value -> 'b value) -> ('a -> 'b) value
and 'a expr =
    Var : string * 'a typ -> 'a expr
  | App : ('a -> 'b) expr * 'a expr -> 'b expr
  | Lam : string * 'a typ * 'b expr -> ('a -> 'b) expr
  | Val : 'a value -> 'a expr
  | Plus : int expr * int expr -> int expr
  | Mult : int expr * int expr -> int expr
  | Ite : bool expr * 'a expr * 'a expr -> 'a expr


In [325]:
let rec typ_eq : type a b. a typ -> b typ -> (a, b) eql option =
  fun t1 t2 -> match t1, t2 with
  | TInt, TInt -> Some Refl
  | TBool, TBool -> Some Refl
  | TLam (t1,t2), TLam (u1,u2) -> 
      begin match typ_eq t1 u1, typ_eq t2 u2 with
      | Some Refl, Some Refl -> Some Refl
      | _ -> None
      end
  | _ -> None

val typ_eq : 'a typ -> 'b typ -> ('a, 'b) eql option = <fun>


In [326]:
type env =
  | Empty : env
  | Extend : string * 'a typ * 'a value * env -> env

type env = Empty : env | Extend : string * 'a typ * 'a value * env -> env


## Evaluation

```ocaml
eval : a expr -> a value
```

* Limitations:
  + Evaluation is defined only on closed terms.
  + Type checking for variables use done at runtime.

In [352]:
let rec eval : type a. env -> a expr -> a value = 
  fun env e -> match e with
  | Var (s,t) -> 
     begin match env with
      | Empty -> failwith "not a closed term"
      | Extend (s',t',v,env') ->
          if s = s' then 
            match typ_eq t t' with
            | Some Refl -> v
            | None -> failwith "types don't match"
          else eval env' e
     end
  | App (e1,e2) -> 
    let VLam f = eval env e1 in
    f (eval env e2)
  | Lam (s,t,e) -> 
      VLam (fun v -> eval (Extend (s, t, v, env)) e)
  | Val v -> v
  | Plus (e1, e2) ->
    let VInt i1, VInt i2 = eval env e1, eval env e2 in
    VInt (i1 + i2)
  | Mult (e1, e2) ->
    let VInt i1, VInt i2 = eval env e1, eval env e2 in
    VInt (i1 * i2)
  | Ite (p,e1,e2) ->
    let VBool b = eval env p in
    if b then eval env e1 else eval env e2
    
let eval e = eval Empty e

val eval : env -> 'a expr -> 'a value = <fun>


val eval : 'a expr -> 'a value = <fun>


## Typing catches errors

In [345]:
eval @@ App (Lam ("x",TInt,Val(VInt 0)), (Val (VInt 10)))

- : int value = VInt 0


In [354]:
eval @@ App (Lam ("x",TInt,Val(VInt 0)), (Val (VBool 10)))

error: compile_error

## Runtime errors

In [355]:
eval @@ App (Lam ("x",TBool, Plus (Val (VInt 10), Var ("x", TInt))), Val (VBool true))

error: runtime_error

In [356]:
eval @@ Var ("x", TInt)

error: runtime_error

## Fixing the runtime errors

* Use the host language (OCaml) features for encoding the features in the object language (lambda calculus).
  + The feature we use here is abstraction.
  + Use `fun (x : TInt) -> e` to encode `Lam "x" TInt e`

In [372]:
type 'a value =
  | VInt : int -> int value
  | VBool : bool -> bool value
  | VLam : ('a expr -> 'b expr) -> ('a -> 'b) value
  
and 'a expr =
  | App : ('a -> 'b) expr * 'a expr -> 'b expr
  | Val : 'a value -> 'a expr
  | Plus : int expr * int expr -> int expr
  | Mult : int expr * int expr -> int expr
  | Ite : bool expr * 'a expr * 'a expr -> 'a expr

type 'a value =
    VInt : int -> int value
  | VBool : bool -> bool value
  | VLam : ('a expr -> 'b expr) -> ('a -> 'b) value
and 'a expr =
    App : ('a -> 'b) expr * 'a expr -> 'b expr
  | Val : 'a value -> 'a expr
  | Plus : int expr * int expr -> int expr
  | Mult : int expr * int expr -> int expr
  | Ite : bool expr * 'a expr * 'a expr -> 'a expr


## Higher Order Abstract Syntax

* This technique is known as Higher-Order Abstract Syntax (HOAS).
* Downside is that we can no longer **look inside** the abstraction.
  + In the term `Lam f`, we cannot do anything on `f` except apply it. 
  + Cannot implement full-beta reduction for example 
  + or do code transformation inside the body of lambda. 

## Evaluator

In [375]:
let rec eval : type a. a expr -> a value = 
  fun e -> match e with
  | App (e1,e2) -> 
    let VLam f = eval e1 in
    eval (f (Val (eval e2)))
  | Val v -> v
  | Plus (e1, e2) ->
    let VInt i1, VInt i2 = eval e1, eval e2 in
    VInt (i1 + i2)
  | Mult (e1, e2) ->
    let VInt i1, VInt i2 = eval e1, eval e2 in
    VInt (i1 * i2)
  | Ite (p,e1,e2) ->
    let VBool b = eval p in
    if b then eval e1 else eval e2

val eval : 'a expr -> 'a value = <fun>


## Static errors remain static

In [378]:
eval @@ App (Val (VLam (fun x -> Val(VInt 0))), (Val (VInt 10)))

- : int value = VInt 0


In [381]:
eval @@ App (Val (VLam (fun x -> Val(VInt 0))), (Val (VBool 10)))

error: compile_error

## Dynamic errors also become static

In [382]:
eval @@ App (Val (VLam (fun (x : bool expr) -> Plus (Val (VInt 10), x))), Val (VBool true))

error: compile_error